In [1]:
from datetime import timedelta
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

query = "SELECT * FROM public.backup_table;"
df = pd.read_sql(query, con=engine)

# Convert date columns to datetime format
df['policy start date'] = pd.to_datetime(df['policy start date'], errors='coerce')
df['policy end date'] = pd.to_datetime(df['policy end date'], errors='coerce')

if 'old policy no' not in df.columns:
    df['old policy no'] = ''
df['old policy no'] = df['old policy no'].fillna('').astype(str)

len(df)

2307835

In [ ]:
def map_old_policy_no(group):
    group = group.sort_values(by='policy start date')
    for i in range(len(group) - 1):
        current_policy = group.iloc[i]
        next_policy = group.iloc[i + 1]
        # Map the old policy number if not already set
        if next_policy['old policy no'] in ['', 'None']:
            if next_policy['policy start date'] == current_policy['policy end date'] + timedelta(days=1):
                group.loc[next_policy.name, 'old policy no'] = current_policy['policy no']
            elif next_policy['policy start date'] > current_policy['policy end date'] + timedelta(days=1):
                group.loc[next_policy.name, 'old policy no'] = current_policy['policy no']
    return group

# Apply the map_old_policy_no function to each group based on "Trim Policy No"
df_cleaned = df.groupby('Trim Policy No').apply(map_old_policy_no).reset_index(drop=True)